# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"  width="80" /> IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Semestre 2022-2**<br>
**Profesora:** Paula Aguirre <br>

## Clase 25: DBSCAN



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import matplotlib.pylab as plt
plt.rcParams['font.size'] = 16

## 1. Datos de entrada

In [3]:
stgo=gpd.read_file('santiago_urbano.gpkg')
crashes=gpd.read_file('choques_santiago.gpkg')
crashes.head()

AttributeError: module 'geopandas' has no attribute 'read_file'

In [ ]:
crashes.info()

In [ ]:
ax = stgo.plot(figsize=(18, 16), color='#EFEFEF', edgecolor='#444444')
crashes.plot(ax=ax, color='red', markersize=8)
plt.title("2018 Road Crashs - Santiago, Chile");

### ¿En qué sistema de coordenadas es recomendable operar los datos?

In [ ]:
crashes=crashes.to_crs(epsg=32719)
stgo=stgo.to_crs(epsg=32719)

In [ ]:
crashes.head()

In [ ]:
crashes['Easting']=crashes.geometry.x
crashes['Northing']=crashes.geometry.y

In [ ]:
X=crashes[['Easting','Northing']]
X

In [ ]:
# Ejecutar DBSCAN
#fit_predict: predice las etiquetas de cada observación
#clusters = DBSCAN(eps=200, min_samples=20).fit_predict(X)
db= DBSCAN(eps=200, min_samples=20).fit(X)

In [ ]:
clusters=db.labels_
len(clusters)

In [ ]:
#asignamos etiquetas de clusters a una nueva columna del dataframe
crashes['cluster_lbl']=clusters
crashes.head(50)

In [ ]:
len(crashes['cluster_lbl'].unique())

In [ ]:
# Ruido
noise = crashes[crashes['cluster_lbl']==-1] #label=-1

#Clusters
cores = crashes[crashes['cluster_lbl']!=-1]#label<>-1

In [ ]:
vals = np.linspace(0,1,144)
np.random.seed(111) 
np.random.shuffle(vals)
cmap = plt.cm.colors.ListedColormap(plt.cm.tab20b(vals))

ax = stgo.plot(figsize=(18, 16), color='white', edgecolor='gray')
cores.plot(ax=ax, column='cluster_lbl',categorical=True, cmap=cmap,markersize=8)
noise.plot(ax=ax, color='gray', markersize=4,alpha=0.4)

plt.title("Clusters de Choques - Santiago, Chile")
plt.axis('off');

### ¿Qué pasa si agregamos otra variable al análisis? Por ejemplo, el número de heridos graves en cada choque.

In [ ]:
X=crashes[['Easting','Northing','Graves']]
X=StandardScaler().fit_transform(X)
X

In [ ]:
crashes.describe()

In [ ]:
# Compute DBSCAN
db2 = DBSCAN(eps=0.02, min_samples=20).fit(X)
crashes['cluster_lbl_2']=db2.labels_


In [ ]:
# Ruido
noise = crashes[crashes['cluster_lbl_2']==-1]
cores = crashes[crashes['cluster_lbl_2']!=-1]

vals = np.linspace(0,1,359)
np.random.seed(111) 
np.random.shuffle(vals)
cmap = plt.cm.colors.ListedColormap(plt.cm.tab20b(vals))

ax = stgo.plot(figsize=(18, 16), color='white', edgecolor='gray')
cores.plot(ax=ax, column='cluster_lbl_2',categorical=True, cmap=cmap,markersize=20)
noise.plot(ax=ax, color='gray', markersize=4,alpha=0.4)

plt.title("2018 Choques - Santiago, Chile")
plt.axis('off');

In [ ]:
crashes.groupby('cluster_lbl').mean()